# Лабораторная работа 2
## Решение СЛАУ прямыми и итерационными методами
Выполнил: Шумаков Иван Б01-009

Прямой метод - метод Холецкого

Итерационный - метод верхней релаксации

Анализ проводился по первой норме

### Начальные данные
Анализ проводился для матрицы пункта `л)`
$$ \begin{cases}
    a_{11} x_1 + a_{12} x_2 + \dots + a_{1n} x_n = f_1 \\
    \dots                                              \\
    a_{n1} x_1 + a_{n2} x_2 + \dots + a_{nn} x_n = f_n \\   
   \end{cases}  $$
$$ n = 20, a_{ii} = 10, a_{ij} = 1 / (i + j), f_i = 1 / i $$

### Итерационный метод
Общая формула для метода верхних релаксаций:
$$  x_j^{i + 1} = -\tau \sum_{k = 1}^{j - 1} \frac{a_{jk}}{a_{jj}}x_k^{i + 1}
    + (1 - \tau)x_j^i - \tau \sum_{k = j + 1}^n \frac{a_{jk}}{a_{jj}}k_k^i
    + \tau \frac{f_i}{a_{jj}} $$

Критерий останова для метода верхних релаксаций:
$$ ||x^{k + 1} - x ^k|| < \epsilon   $$
$$ \epsilon = 0.01 $$

Невязка на каждом шаге выражается формулой:
$$ r_k = || A x^k - f || $$

### Прямой метод
В качестве прямого метода был выбран метод Холецкого:
$$ A = L L^T $$
$$  \begin{cases}
        L y = f \\
        L^T x = y
    \end{cases} $$

Коэффициенты матрицы L выражаются как:
$$ l_{ij} = \frac{1}{l_{jj}} (a_{ij} - \sum_{k = 1}^{j - 1} l_{ik} j_{jk}) $$
$$ l_{ii} = \sqrt{a_{ii} - \sum_{k = 1}^{i - 1} l_{ik}^2} $$

### Анализ матрицы
Число обусловленности:
$$ \mu = ||A||A^{-1}|| $$

Поиск $\lambda_{min} \lambda_{max}$ степенным методом:

$$  y_k = A y_{k - 1} \\
    \lambda_{max} = \frac{||y_k||}{||y_{k - 1}||} $$

Гуманитарное объяснение метода:

Представим этот итерационный процесс в базисе собственных векторов. Пусть начальный вектор имеет компоненты по всем осям, тогда на каждой итерации его составляющие по каждому из базисных векторов будут умножаться на соответствующие собственные значения. Таким образом составляющая по базисному вектору с максимальным собственным значением будет расти быстрее остальных(отсюда можно сделать оценку, что сходимость это геом. прогрессия) и в пределе бесконечности $y_k$ будет направлен по базисному вектору с максимальным собственным значением. Поэтому на новой итерации этот ветор будет просто умножаться на максимальное собственное значение.

Для поиска минимального собственного значения необходимо заменить $A -> A^{-1}$ и разделить 1 на получившееся число:

$$  y_k = A^{-1} y_{k - 1} \\
    \lambda_{min} = 1 / \frac{||y_k||}{||y_{k - 1}||} $$

(эти выводы слкдуют из рассуждений выше как обратный процесс преобразования)

Из этого объяснения можно сделать неутешительный вывод: если начальный вектор не имеет компоненты по базисному вектору с максимальным собственным значением, то он не будет к нему сходиться. Таким образом начальных векторов, которые будут выдавть неправильный ответ, бесконечно много.

## Выоды
Ответы, полученные с помощью разных, методов сходятся с разницей во второй - третьей значащих цифрах.
Степенной метод поиска собственных значений сходится с точным решением.

In [3]:
# %load 'Matrix.py'
import numpy as np             
import matplotlib.pyplot as plt

n = 20
diag = 10

class Norm:

    #возвращает максимальную сумму в строке
    def max_sum(matrix):
        max = np.sum(np.abs(matrix[0]))
        for line in matrix:
            cur_sum = np.sum(np.abs(line))
            if cur_sum > max:
                max = cur_sum
        return max

    def __get_first_norm(matrix):
        matr_size = matrix.shape
        if len(matr_size) != 1:
            return Norm.max_sum(matrix)
        else:
            return np.amax(np.abs(matrix))

    def get_norm(matrix):
        return Norm.__get_first_norm(matrix)


class UpperRelax:

    def __init__(self, A, f):
        self.A = A
        self.f = f
        self.teta = 1.5
        self.epsilon = 0.01
        matr_size = A.shape
        if (len(matr_size) != 2) or (matr_size[0] != matr_size[1]):
            print('Incorrect matrix')
        self.n = matr_size[0]
        self.r_arr = np.array([])

    def __get_initial_x(self):
        x = np.copy(self.f)
        return x

    #подсчет невязки
    def __calc_r(self, x):
        r = Norm.get_norm(np.matmul(self.A, x) - self.f)
        self.r_arr = np.append(self.r_arr, r)

    #концы включены
    #х - массив
    def __sum(self, start, end, x, j):
        sum = 0.0
        for k in range(end - start + 1):
            sum += self.A[j, start + k] / self.A[j, j] * x[start + k]
        return sum

    #индексация с 0
    def __iteration(self, x_prev):
        x_new = np.array([])
        for j in range(self.n):
            x_j = (-1) * self.teta * self.__sum(0, j - 1, x_new, j) + (1 - self.teta) * x_prev[j] \
                - self.teta * self.__sum(j + 1, self.n - 1, x_prev, j) + self.teta * self.f[j] / self.A[j, j]
            x_new = np.append(x_new, x_j)
        return x_new
    
    def calculate(self):
        num_of_iter = 0.0
        x_prev = self.__get_initial_x()
        x_new = np.array([])
        prev_error = np.Inf
        curr_error = 0.0
        while True:
            x_new = self.__iteration(x_prev)
            self.__calc_r(x_new)
            curr_error = Norm.get_norm(x_new - x_prev)
            if curr_error < self.epsilon:
                print('конечная разность:', Norm.get_norm(x_new - x_prev))
                break
            else:
                x_prev = x_new
            if prev_error < curr_error:
                print('Warning: on iteration', num_of_iter, 'error was', prev_error, ' and now', curr_error)
            prev_error = curr_error
            num_of_iter += 1
        return x_new


#метод Холецкого
class SqrtMethod:

    def __init__(self, A, f):
        self.A = A
        self.f = f
        self.L = np.full((n, n), 0.0)
        self.L_T = np.full((n, n), 0.0)
        matr_size = A.shape
        if (len(matr_size) != 2) or (matr_size[0] != matr_size[1]):
            print('Incorrect matrix')
        self.n = matr_size[0]
        self.r_arr = np.array([])

    #концы включены
    def __sum(self, end, left_index_1, left_index_2):
        sum = 0.0
        for k in range(end + 1):
            sum += self.L[left_index_1, k] * self.L[left_index_2, k]
        return sum

    def __get_L_elem(self, i, j):
        if i == j:
            return np.sqrt(self.A[i, i] - self.__sum(i - 1, i, i))
        else:
            return (self.A[i, j] - self.__sum(j - 1, j, i)) / self.L[j, j]

    def __fill_L(self):
        for i in range(n):
            for j in range(i + 1):
                self.L[i, j] = self.__get_L_elem(i, j)
        self.L_T = np.copy(self.L)
        self.L_T = np.transpose(self.L_T)
    
    #ytne +1 поскольку при вызове добавляется 1 тк в массиве нумерация с 0
    def __sum_y(self, num_of_iter, cur_index, y):
        sum = 0.0
        for k in range(num_of_iter):
            sum += self.L[cur_index, k] * y[k] 
        return sum

    #решение уравнения Ly=f (обратный ход) 
    #L - нижне-треугольная
    def find_y(self):
        y = np.array([])
        for i in range(self.n):
            y_i = (self.f[i] - self.__sum_y(i - 1 + 1, i, y)) / self.L[i, i]   
            y = np.append(y, y_i)
        return y

    #тут не надо + 1 (количество итераций достаточно)
    def __sum_x(self, num_of_iter, cur_index, x):
        sum = 0.0
        for p in range(num_of_iter):
            sum += self.L_T[cur_index, cur_index + p + 1] * x[cur_index + p + 1]
        return sum

    #решение уравнения L^T x = y
    def find_x(self, y):
        x = np.full(self.n, 0.0)
        for k in range(self.n):
            x[self.n - k - 1] = (y[n - k - 1] - self.__sum_x(k, self.n - k - 1, x)) \
                / self.L_T[n - k - 1, n - k - 1]
        return x

    def calculate(self):
        self.__fill_L()
        y = self.find_y()
        x = self.find_x(y)
        return x


#скалярное произведение с Г=А xAy 
def scalar_by_A(x, A, y):
    first = np.matmul(x, A)
    return np.matmul(first, y)

#подсчет минимального и максимального собственного значения
#на википедии неправильный метод
def calc_lambda(A):
    num_of_iter = 200
    y_new = np.copy(A[0])
    y_prev = np.copy(A[0])
    for i in range(num_of_iter):
        y_prev = y_new
        mult = np.matmul(A, y_prev) 
        y_new = mult                                      #/ Norm.get_norm(mult) #деление на норму помогает избежать переполнения
    return  Norm.get_norm(y_new) / Norm.get_norm(y_prev)  #scalar_by_A(np.transpose(y_new), A, y_new) / scalar_by_A(np.transpose(y_prev), A, y_prev) 

#если перемножить получится единичная матрица с точностью до 10^-18
def determine_koef(A):
    rev_A = np.linalg.inv(A)
    return Norm.get_norm(A) * Norm.get_norm(rev_A)

def get_line(line_num):
    line = np.array([])
    for j in range(n):
        if line_num == j:
            line = np.append(line, diag)
        else:    
            line = np.append(line, 1 / (line_num + j + 2))
    return line

def generate_matrix():
    matr = get_line(0)
    for i in range(n - 1):
        line = get_line(i + 1)
        matr = np.vstack((matr, line))
    return matr

def generate_rhs():
    rhs = np.array([])
    for i in range(n):
        rhs = np.append(rhs, 1 / (i + 1))
    return np.transpose(rhs)

def iteration_method(A, f):
    method = UpperRelax(A, f)
    method.calculate()

def main():
    A = generate_matrix()
    f = generate_rhs()
    method = UpperRelax(A, f)
    ans_iter = method.calculate()
    print('Ответ методом верхних релаксаций:', ans_iter)
    straight_method = SqrtMethod(A, f)
    ans_straight = straight_method.calculate()
    print('Ответ методом Холецкого:', ans_straight)
    print('Невязка:', method.r_arr)
    print('')
    print('Число обусловленности', determine_koef(A))
    print('Максиммальное собственное значение:', calc_lambda(A))
    print('Минимальное собственное значение через A^-1:', 1 / calc_lambda(np.linalg.inv(A)))
    #print('Минимальное собственное значение через A^T:', calc_lambda(np.transpose(A)))
    num, vec =  np.linalg.eigh(A)
    print('')
    print('Реальные собственные значения для :', num)

if __name__ == '__main__':
    main()

конечная разность: 0.008235196572854153
Ответ методом верхних релаксаций: [0.09347738 0.04482422 0.02913087 0.02144176 0.01689773 0.01390572
 0.01179117 0.01022005 0.00900831 0.0080463  0.00726471 0.0066176
 0.00607335 0.00560947 0.00520956 0.00486139 0.00455563 0.00428505
 0.00404399 0.0038279 ]
Ответ методом Холецкого: [0.09615746 0.04478375 0.02874755 0.02101576 0.0164943  0.01353947
 0.01146294 0.00992677 0.00874602 0.00781115 0.00705327 0.00642689
 0.0059008  0.00545292 0.00506717 0.00473155 0.00443698 0.00417641
 0.00394434 0.00373636]
Невязка: [5.3319564  2.94257316 1.591187   0.84382867 0.43901868 0.22405194
 0.11208979 0.05490131 0.02627231]

Число обусловленности 1.4526954762585125
Максиммальное собственное значение: 11.300128398304816
Минимальное собственное значение через A^-1: 9.658552341348045

Реальные собственные значения для : [ 9.65779712  9.8117211   9.86808709  9.89787972  9.91645175  9.92919102
  9.93849622  9.94560315  9.95121536  9.95576376  9.95952748  9.9626955